# Rastreador de componentes y eventos de logs.
En este Notebook permite generar un fichero JSON con los diferentes componentes, eventos y descripciones que existen en un log de Moodle. Los logs deben descargarse en Moodle en formato CSV.
También permite actualizar a partir de un fichero JSON para añadir nuevos elementos del log con otros ficheros CSV.

In [1]:
from tkinter import filedialog
import csv
import tkinter as tk
import json
import re

# Carga de ficheros CSV y JSON (opcional)
En esta celda permite seleccionar los ficheros CSV y opcionalmente un fichero .json donde se guardan los componentes y eventos generados anteriormente. En ambos casos se podran seleccionar a partir de una venta de selección de ficheros.
Los CSV se puede elegir varios a la vez. Si no tenemos fichero JSON, damos en el botón de cancelar. <br>
**OJO** Tener cuidado al elegir los CSV y JSON esten en el mismo idioma.<br><br>
Fuente del código de la celda: https://stackoverflow.com/a/42398049

In [2]:

# Create Tk root
root = tk.Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

csvFiles = filedialog.askopenfilename(multiple=True,filetypes=[("Ficheros CSV",".csv")],title="Selecciona los ficheros CSV")

jsonFile=filedialog.askopenfilename(filetypes=[("Fichero JSON",".json")],title="Selecciona el fichero JSON donde se guarda datos anteriores, si no tienes dale a cancelar")

%gui tk

# Carga del fichero JSON
Cargamos el fichero JSON seleccionado en la anterior ventana, si no se ha seleccionado ninguno creamos un nuevo diccionario.

In [3]:
#Cargamos los componentes y eventos si se ha seleccionado fichero json

if len(jsonFile)==0:
    print("No se ha seleccionado un fichero JSON, se crea una nueva estructura de datos.")
    CM={}
else:
    with open(jsonFile,"r",encoding="utf-8") as file:
        CM=json.load(file)
        print("Cargado el JSON", jsonFile)

No se ha seleccionado un fichero JSON, se crea una nueva estructura de datos.


# Parseo de las descripciones y almacenamiento de los componentes y eventos.
Iteramos por cada fichero CSV seleccionado cogemos las columnas de Component, Nombre de evento y Descripción. Estas columnas deben ser los índices 4,5 y 6 empezando desde 0 respectivamente. Si hay una actualización de Moodle y cambia el orden hay que cambiar también en el código.<br><br>
Las descripciones eliminamos el contenido dentro de las comillas simples y tambien borramos todos los numeros que aparecen.<br><br>
Imprimimos las descripciones que hay mas de una diferente con el mismo componente y evento asociado.


In [4]:
for csvFile in csvFiles: #por cada fichero seleccionado, lo parsemamos uno a uno.
    with open(csvFile,encoding='utf-8') as file:
        #reader = csv.DictReader(file, delimiter=',') #Lector de CSV con acceso a una columna por el nombre de encabezado
        reader=csv.reader(file, delimiter=',') #Lector de CSV con acceso a una comlmna a traves del indice.
        for row in reader:
            component=row[4]#Se supone que el componente sencuentra por la cuarta columna empezando por 0
            event=row[5]
            description=row[6]
            
            descripNotNumber=re.sub("'[^']+'","''",description) #eliminamos el texto que esta entre comillas simples
            descripNotNumber=re.sub("\d+", "", descripNotNumber) #eliminamos los numeros
            
            #el setdefault, si existe la key devuelve el valor y si no se crea segun el segundo parametro y lo devuelve.
            descriptions=CM.setdefault(component,{}).setdefault(event,[]) 
            if(descripNotNumber not in descriptions):
                descriptions.append(descripNotNumber)
                if len(descriptions)>1:
                    print("Componente y evento con varias descripciones diferentes:")
                    print("\tComponent: ",component,"\tEvent:",event,"\tDescriptions: ",descriptions)
            

Componente y evento con varias descripciones diferentes:
	Component:  System 	Event: Course viewed 	Descriptions:  ["The user with id '' viewed the course with id ''.", "The user with id '' viewed the section number '' of the course with id ''."]


# Se guarda el resultado como fichero JSON


In [5]:
#Guardamos como JSON
#mantenemos el nombre original del json y sino los llamamos componentes y eventos
filename=  jsonFile if len(jsonFile)>0 else "Componentes y eventos.json" 
with open(filename, 'w',encoding='utf-8' ) as f:
    json.dump( CM, f,indent=4,sort_keys=True,ensure_ascii=False )
    print("Guardado como JSON", filename)

Guardado como JSON Componentes y eventos.json
